<a href="https://colab.research.google.com/github/Tauheed-a21/machine-learning/blob/main/RNN_mov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import random
import re
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

MAX_VOCAB_SIZE = 30000
MAX_SEQUENCE_LENGTH = 300
EMBEDDING_DIM = 128
BATCH_SIZE = 64
EPOCHS = 10
VALIDATION_SPLIT = 0.1
TEST_SIZE = 0.1
RNN_TYPE = "SimpleRNN"
MODEL_SAVE_PATH = "rnn_imdb_best.h5"
TOKENIZER_PATH = "tokenizer.json"
SEED = 42

os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

def load_dataset(path="IMDB Dataset.csv"):
    df = pd.read_csv(path)
    df.columns = [c.lower() for c in df.columns]
    df = df[['review', 'sentiment']].dropna()
    df['label'] = df['sentiment'].map(lambda x: 1 if str(x).strip().lower().startswith('pos') else 0)
    return df[['review', 'label']]

def clean_text(text):
    text = re.sub(r'<br\s*/?>', ' ', text)
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r"[^a-zA-Z0-9' ]", ' ', text)
    text = re.sub(r"\s+", ' ', text)
    return text.strip().lower()

def prepare_tokenizer(texts, max_words=MAX_VOCAB_SIZE):
    tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
    tokenizer.fit_on_texts(texts)
    return tokenizer

def sequences_from_text(tokenizer, texts, maxlen=MAX_SEQUENCE_LENGTH):
    seqs = tokenizer.texts_to_sequences(texts)
    return pad_sequences(seqs, maxlen=maxlen, padding='post', truncating='post')

def build_model(rnn_type=RNN_TYPE, vocab_size=MAX_VOCAB_SIZE, embedding_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_length))

    if rnn_type == 'SimpleRNN':
        model.add(SimpleRNN(128))
    elif rnn_type == 'LSTM':
        model.add(LSTM(128))
    elif rnn_type == 'BidirectionalLSTM':
        model.add(Bidirectional(LSTM(128)))
    elif rnn_type == 'BidirectionalSimpleRNN':
        model.add(Bidirectional(SimpleRNN(128)))

    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def main(dataset_path='IMDB Dataset.csv'):
    df = load_dataset(dataset_path)
    df['clean_review'] = df['review'].apply(clean_text)
    train_val_df, test_df = train_test_split(df, test_size=TEST_SIZE, random_state=SEED, stratify=df['label'])

    tokenizer = prepare_tokenizer(train_val_df['clean_review'].tolist(), max_words=MAX_VOCAB_SIZE)
    vocab_size = min(MAX_VOCAB_SIZE, len(tokenizer.word_index) + 1)

    X_train_val = sequences_from_text(tokenizer, train_val_df['clean_review'].tolist(), MAX_SEQUENCE_LENGTH)
    y_train_val = train_val_df['label'].values
    X_test = sequences_from_text(tokenizer, test_df['clean_review'].tolist(), MAX_SEQUENCE_LENGTH)
    y_test = test_df['label'].values

    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=VALIDATION_SPLIT, random_state=SEED, stratify=y_train_val)

    model = build_model(rnn_type=RNN_TYPE, vocab_size=vocab_size, embedding_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True),
        ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_loss', save_best_only=True)
    ]

    model.fit(
        X_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(X_val, y_val),
        callbacks=callbacks
    )

    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
    print(f'Test loss: {test_loss:.4f}  Test accuracy: {test_acc:.4f}')

    y_pred = (model.predict(X_test) >= 0.5).astype(int).reshape(-1)
    print(classification_report(y_test, y_pred, digits=4))

if __name__ == '__main__':
    main('IMDB Dataset.csv')

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


633/633 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.4931 - loss: 0.7285

633/633 ━━━━━━━━━━━━━━━━━━━━ 124s 192ms/step - accuracy: 0.4931 - loss: 0.7285 - val_accuracy: 0.5031 - val_loss: 0.6932
Epoch 2/10
633/633 ━━━━━━━━━━━━━━━━━━━━ 115s 182ms/step - accuracy: 0.5028 - loss: 0.6990 - val_accuracy: 0.4898 - val_loss: 0.6942
Epoch 3/10
633/633 ━━━━━━━━━━━━━━━━━━━━ 122s 193ms/step - accuracy: 0.4988 - loss: 0.6953 - val_accuracy: 0.5000 - val_loss: 0.6941
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.4965 - loss: 0.6943
Test loss: 0.6938  Test accuracy: 0.5044
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step
              precision    recall  f1-score   support

           0     0.5049    0.4564    0.4794      2500
           1     0.5040    0.5524    0.5271      2500

    accuracy                         0.5044      5000
   macro avg     0.5044    0.5044    0.5033      5000
weighted avg     0.5044    0.5044    0.5033      5000

